# Forecasting Inbound Calls by Hour

This notebook implements a forecasting model for inbound calls using LSTM neural networks. It includes data preprocessing, anomaly detection, feature engineering, model training, evaluation, and future predictions. Additionally, it calculates required operators using ErlangC for workforce management.

Author: [Your Name]
Date: October 23, 2025

## Importing Libraries

Import necessary libraries for data manipulation, visualization, machine learning, and time series analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10, 5)
from sklearn.ensemble import RandomForestRegressor
from warnings import catch_warnings, filterwarnings
from statsmodels.tsa.stattools import adfuller, kpss
import seaborn as sns
import datetime
from sklearn.metrics import mean_squared_error, mean_squared_log_error, mean_absolute_error
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import randint

# Neural network libraries
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# For anomaly detection
from sklearn.ensemble import IsolationForest

## Data Import and Preprocessing

Load the dataset and perform initial transformations.

In [ ]:
# Load the data
data = pd.read_csv('tmp001 05112024.csv', sep=';', header=0, parse_dates=True)

# Replace commas with dots and convert to float
data[['IS_LUNCH', 'IS_WORKTIME', 'IS_MORNING']] = data[['IS_LUNCH', 'IS_WORKTIME', 'IS_MORNING']].replace(',', '.', regex=True).astype(float)

# Print data types to verify
print(data[['IS_LUNCH', 'IS_WORKTIME', 'IS_MORNING']].dtypes)

In [ ]:
# Convert columns to appropriate formats
data['MONTH'] = pd.to_datetime(data['MONTH'] + '-01')  # Convert month to datetime
data['DATESTART'] = pd.to_datetime(data['DATESTART'], dayfirst=True)  # Convert date to datetime
data['RAZREZ'] = pd.to_timedelta(data['RAZREZ'] + ':00')  # Convert time to timedelta

# Create full timestamp
data['TIMESTAMP'] = data['DATESTART'] + data['RAZREZ']
data.head(2)

In [ ]:
# Filter records starting from 2022-04-01
Hourly_calls = data[(data['DATESTART'] >= pd.to_datetime('2022-04-01'))]
Hourly_calls

In [ ]:
# Set TIMESTAMP as index
Hourly_calls.set_index('TIMESTAMP', inplace=True)
Hourly_calls.head(2)

## Anomaly Detection and Correction

Detect and handle anomalies in the call volume data using Isolation Forest.

In [ ]:
# Prepare data for anomaly detection
anomaly_analysis = Hourly_calls.copy()

# Plot original call volume
anomaly_analysis['CNT_CALLS'].plot(figsize=(12, 6))
plt.title('Call Volume')
plt.show()

# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(anomaly_analysis[['CNT_CALLS']])

# Train Isolation Forest model
model = IsolationForest(contamination=0.001)
model.fit(scaled_data)

# Predict anomalies
anomaly_analysis['anomaly'] = model.predict(scaled_data)
anomaly_analysis['anomaly'] = anomaly_analysis['anomaly'].map({1: 'normal', -1: 'anomaly'})

# Visualize anomalies
plt.figure(figsize=(12, 6))
plt.plot(anomaly_analysis.index, anomaly_analysis['CNT_CALLS'], label='Number of calls')
plt.scatter(anomaly_analysis.index[anomaly_analysis['anomaly'] == 'anomaly'], 
            anomaly_analysis['CNT_CALLS'][anomaly_analysis['anomaly'] == 'anomaly'], 
            color='red', label='Anomalies')
plt.legend()
plt.title('Anomalies in Call Volume Data')
plt.show()

# Replace anomalies with median
median_value = anomaly_analysis['CNT_CALLS'].median()
anomaly_analysis.loc[anomaly_analysis['anomaly'] == 'anomaly', 'CNT_CALLS'] = median_value

# Plot corrected data
plt.figure(figsize=(12, 6))
plt.plot(anomaly_analysis.index, anomaly_analysis['CNT_CALLS'], label='Corrected Call Volume')
plt.title('Corrected Call Volume Data')
plt.show()

Hourly_calls = anomaly_analysis

## Feature Engineering

Create new features such as weekday dummies, time interval indicators, billing days, lag features, and rolling statistics.

In [ ]:
# Add weekday feature (Monday=0, ..., Sunday=6)
Hourly_calls['weekday'] = Hourly_calls.index.weekday

# One-hot encode weekday
weekday_dummies = pd.get_dummies(Hourly_calls['weekday'], prefix='weekday')
weekday_dummies = weekday_dummies.astype(int)
Hourly_calls = Hourly_calls.join(weekday_dummies, how='left')
Hourly_calls.head(2)

In [ ]:
# Create dummy variables for unique time intervals
Hourly_calls['RAZREZ'] = pd.to_timedelta(Hourly_calls['RAZREZ'])
unique_razrez = Hourly_calls['RAZREZ'].unique()

for razrez in unique_razrez:
    col_name = f'IS_{str(razrez).replace(" ", "_").replace("days", "d").replace(":", "_")}'
    Hourly_calls[col_name] = Hourly_calls['RAZREZ'].apply(lambda x: 1 if x == razrez else 0)

Hourly_calls.head(2)

In [ ]:
# Functions to identify critical and billing days
def is_critical_day(date):
    """Check if the day is a critical day (pre-billing)."""
    return 1 if date.day in [3, 4, 10, 13, 14, 23, 24] else 0

def is_billing_day(date):
    """Check if the day is a billing day."""
    return 1 if date.day in [5, 15, 25] else 0

# Add critical_day and billing_day features
Hourly_calls['critical_day'] = Hourly_calls['DATESTART'].apply(is_critical_day)
Hourly_calls['billing_day'] = Hourly_calls['DATESTART'].apply(is_billing_day)
Hourly_calls.head(2)

In [ ]:
# Add lag features (previous 10 periods)
for i in range(1, 11):
    Hourly_calls[f'Inq_{i}'] = Hourly_calls['CNT_CALLS'].shift(i)

Hourly_calls.head(2)

In [ ]:
# Calculate rolling mean (7 periods) and std (3 periods)
Hourly_calls['Inq_mean_7'] = Hourly_calls['Inq_1'].rolling(window=7).mean()
Hourly_calls['Inq_std_3'] = Hourly_calls['Inq_1'].rolling(window=3).std()
Hourly_calls.head(2)

## Data Preparation for Modeling

Prepare features and target for the LSTM model.

In [ ]:
# Drop unnecessary columns and fill NaNs
final_df = Hourly_calls.drop(columns=[
    'weekday', 'DATESTART', 'MONTH', 'RAZREZ', 'anomaly',
    'IS_0_d_22_00_00', 'IS_0_d_00_00_00', 'IS_0_d_00_30_00',
    'IS_0_d_01_00_00', 'IS_0_d_01_30_00', 'IS_0_d_02_30_00',
    'IS_0_d_03_00_00', 'IS_0_d_04_00_00', 'IS_0_d_05_30_00',
    'IS_0_d_06_00_00', 'IS_0_d_06_30_00', 'IS_0_d_07_00_00',
    'IS_0_d_07_30_00', 'IS_0_d_22_30_00', 'IS_0_d_23_00_00',
    'IS_0_d_23_30_00', 'IS_0_d_04_30_00', 'IS_0_d_03_30_00',
    'IS_0_d_02_00_00', 'IS_0_d_05_00_00'
]).fillna(0)

final_df.head(2)

In [ ]:
# Separate target and features
y = final_df['CNT_CALLS']
X = final_df.drop(columns=['CNT_CALLS'])

# Scale numerical features
scaler_x = StandardScaler()
numerical_columns = ['Inq_1', 'Inq_2', 'Inq_3', 'Inq_4', 'Inq_5', 'Inq_6',
                     'Inq_7', 'Inq_8', 'Inq_9', 'Inq_10', 'Inq_mean_7', 'Inq_std_3']
X[numerical_columns] = scaler_x.fit_transform(X[numerical_columns])

X_new = X.values

# Scale target
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_new, y_scaled, test_size=0.2, random_state=42)

# Reshape for LSTM
time_steps = 1
X_train = np.reshape(X_train, (X_train.shape[0], time_steps, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], time_steps, X_test.shape[1]))

## LSTM Model Definition and Training

Define and train an LSTM model for time series forecasting.

In [ ]:
# Define LSTM model
model = Sequential()
model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=300, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=300))
model.add(Dense(units=1))

# Compile model
learning_rate = 0.00001
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.summary()

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

# Plot training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['mae'], label='MAE (Train)')
plt.plot(history.history['val_mae'], label='MAE (Validation)')
plt.title('Mean Absolute Error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Loss (Train)')
plt.plot(history.history['val_loss'], label='Loss (Validation)')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

## Model Evaluation

Evaluate the model on train and test sets using MAE, RMSE, and R².

In [ ]:
# Inverse scale predictions and targets
y_train_original = scaler_y.inverse_transform(y_train)
y_test_original = scaler_y.inverse_transform(y_test)

# Train predictions
y_train_pred = model.predict(X_train)
y_train_pred_original = scaler_y.inverse_transform(y_train_pred)

# Test predictions
y_test_pred = model.predict(X_test)
y_test_pred_original = scaler_y.inverse_transform(y_test_pred)

# Calculate metrics
train_mae = mean_absolute_error(y_train_original, y_train_pred_original)
train_rmse = np.sqrt(mean_squared_error(y_train_original, y_train_pred_original))
train_r2 = r2_score(y_train_original, y_train_pred_original)
print("Train MAE: ", train_mae)
print('Train RMSE:', train_rmse)
print("Train R²:", train_r2)

test_mae = mean_absolute_error(y_test_original, y_test_pred_original)
test_rmse = np.sqrt(mean_squared_error(y_test_original, y_test_pred_original))
test_r2 = r2_score(y_test_original, y_test_pred_original)
print("Test MAE: ", test_mae)
print('Test RMSE:', test_rmse)
print("Test R²:", test_r2)

In [ ]:
# Prepare DataFrames for visualization
train_rf = pd.DataFrame({'train': y_train_original.ravel(), 'train_pred': y_train_pred_original.ravel()})
predictions_rf = pd.DataFrame({'y_test': y_test_original.ravel(), 'y_test_pred': y_test_pred_original.ravel()})

# Plot test predictions
plt.figure(figsize=(20, 6))
plt.plot(predictions_rf[['y_test', 'y_test_pred']].iloc[4:260].sort_index(), label=['Test', 'Predictions'])
plt.legend(['Test', 'Predictions'])
plt.show()

In [ ]:
# Residuals plot
predictions_rf['residuals'] = predictions_rf['y_test'] - predictions_rf['y_test_pred']
sns.scatterplot(data=predictions_rf, x="y_test_pred", y="residuals")
plt.axhline(y=0)

## Future Predictions

Generate features for future timestamps and predict call volumes.

In [ ]:
# Generate future time range
start_time = (final_df.index[-1] + pd.Timedelta(minutes=30)).to_pydatetime()
end_time = start_time + pd.Timedelta(days=35)
time_range = pd.date_range(start=start_time, end=end_time, freq='30T')

In [ ]:
def generate_features(df):
    """
    Generate time-based features for forecasting.
    
    Parameters:
    df (pd.DataFrame): DataFrame with 'timestamp' column.
    
    Returns:
    pd.DataFrame: DataFrame with added features.
    """
    df['hour'] = df['timestamp'].dt.hour
    df['weekday'] = df['timestamp'].dt.weekday
    df['IS_DAY'] = df['hour'].apply(lambda x: 1 if 8 <= x < 21 else 0)
    df['IS_LUNCH'] = df['hour'].apply(lambda x: 0.2 if 12 <= x < 14 else 0)
    df['IS_WORKTIME'] = df['hour'].apply(lambda x: 0.2 if 8 <= x < 19 else 0)
    df['IS_MORNING'] = df['hour'].apply(lambda x: 0.4 if 8 <= x < 10 else 0)
    df['WORKDAYS'] = df['weekday'].apply(lambda x: 1 if x < 5 else 0)
    df['HOLIDAYS'] = df['weekday'].apply(lambda x: 1 if x >= 5 else 0)
    df['minute'] = df['timestamp'].dt.minute
    df['half_hour'] = df['hour'] * 2 + df['minute'] // 30

    for half_hour in range(48):
        hour = half_hour // 2
        minute = (half_hour % 2) * 30
        col_name = f'IS_0_d_{hour:02d}_{minute:02d}_00'
        df[col_name] = df['half_hour'].apply(lambda x: 1 if x == half_hour else 0)

    weekday_dummies = pd.get_dummies(df['weekday'], prefix='weekday').astype(int)
    df = df.join(weekday_dummies, how='left')

    for i in range(7):
        col_name = f'weekday_{i}'
        if col_name not in df.columns:
            df[col_name] = 0

    df['day'] = df['timestamp'].dt.day
    df['critical_day'] = df['day'].apply(is_critical_day)
    df['billing_day'] = df['day'].apply(is_billing_day)

    df = df.drop(['hour', 'minute', 'half_hour', 'weekday', 'day'], axis=1)
    return df

# Create future DataFrame
future_df = pd.DataFrame(time_range, columns=['timestamp'])
future_df = generate_features(future_df)

# Add lag and rolling columns
for i in range(1, 11):
    future_df[f'Inq_{i}'] = np.nan
future_df['Inq_std_3'] = np.nan
future_df['Inq_mean_7'] = np.nan
future_df['CNT_CALLS'] = np.nan

future_df.set_index('timestamp', inplace=True)

# Reorder columns
columns_order = ['HOLIDAYS', 'WORKDAYS', 'IS_DAY', 'IS_LUNCH', 'IS_WORKTIME', 'IS_MORNING',
                 'CNT_CALLS', 'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3',
                 'weekday_4', 'weekday_5', 'weekday_6',
                 'IS_0_d_08_00_00', 'IS_0_d_08_30_00', 'IS_0_d_09_00_00',
                 'IS_0_d_09_30_00', 'IS_0_d_10_00_00', 'IS_0_d_10_30_00',
                 'IS_0_d_11_00_00', 'IS_0_d_11_30_00', 'IS_0_d_12_00_00',
                 'IS_0_d_12_30_00', 'IS_0_d_13_00_00', 'IS_0_d_13_30_00',
                 'IS_0_d_14_00_00', 'IS_0_d_14_30_00', 'IS_0_d_15_00_00',
                 'IS_0_d_15_30_00', 'IS_0_d_16_00_00', 'IS_0_d_16_30_00',
                 'IS_0_d_17_00_00', 'IS_0_d_17_30_00', 'IS_0_d_18_00_00',
                 'IS_0_d_18_30_00', 'IS_0_d_19_00_00', 'IS_0_d_19_30_00',
                 'IS_0_d_20_00_00', 'IS_0_d_20_30_00', 'IS_0_d_21_00_00',
                 'IS_0_d_21_30_00', 'critical_day', 'billing_day', 'Inq_1', 'Inq_2', 'Inq_3', 'Inq_4', 'Inq_5', 'Inq_6',
                 'Inq_7', 'Inq_8', 'Inq_9', 'Inq_10',
                 'Inq_mean_7', 'Inq_std_3']
future_df = future_df[columns_order]
future_df.head(2)

In [ ]:
# Combine historical and future data
combined_df = pd.concat([final_df, future_df], ignore_index=False)

# Function to calculate lag and rolling features
def calculate_inq_features(df, start_index, predict_index, end_index):
    """
    Calculate lag features and rolling statistics for the DataFrame.
    
    Parameters:
    df (pd.DataFrame): Combined DataFrame.
    start_index (pd.Timestamp): Start index for calculation.
    predict_index (pd.Timestamp): Prediction start index.
    end_index (pd.Timestamp): End index.
    
    Returns:
    pd.DataFrame: Updated DataFrame with features.
    """
    for i in range(1, 11):
        df[f'Inq_{i}'] = df['CNT_CALLS'].shift(i)
    df['Inq_mean_7'] = df['Inq_1'].rolling(window=7).mean()
    df['Inq_std_3'] = df['Inq_1'].rolling(window=3).std()
    return df.loc[start_index:end_index]

# Determine indices
selected_df = combined_df.loc[start_time]
selected_index = combined_df.index.get_loc(selected_df.name)
target_index = max(selected_index - 10, 0)
previous_record_index = combined_df.index[target_index]
start_index = previous_record_index

# Calculate features
calc_df = calculate_inq_features(combined_df, start_index, start_time, end_time)

In [ ]:
# Make predictions iteratively
current_index = pd.to_datetime(start_time)
end_index = pd.to_datetime(end_time)
predicted_calls = []

while current_index <= end_index:
    calc_df.loc[:current_index, numerical_columns] = scaler_x.transform(calc_df.loc[:current_index, numerical_columns])
    features = calc_df.loc[current_index, [
        'HOLIDAYS', 'WORKDAYS', 'IS_DAY', 'IS_LUNCH', 'IS_WORKTIME', 'IS_MORNING',
        'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3',
        'weekday_4', 'weekday_5', 'weekday_6', 'IS_0_d_08_00_00',
        'IS_0_d_08_30_00', 'IS_0_d_09_00_00', 'IS_0_d_09_30_00',
        'IS_0_d_10_00_00', 'IS_0_d_10_30_00', 'IS_0_d_11_00_00',
        'IS_0_d_11_30_00', 'IS_0_d_12_00_00', 'IS_0_d_12_30_00',
        'IS_0_d_13_00_00', 'IS_0_d_13_30_00', 'IS_0_d_14_00_00',
        'IS_0_d_14_30_00', 'IS_0_d_15_00_00', 'IS_0_d_15_30_00',
        'IS_0_d_16_00_00', 'IS_0_d_16_30_00', 'IS_0_d_17_00_00',
        'IS_0_d_17_30_00', 'IS_0_d_18_00_00', 'IS_0_d_18_30_00',
        'IS_0_d_19_00_00', 'IS_0_d_19_30_00', 'IS_0_d_20_00_00',
        'IS_0_d_20_30_00', 'IS_0_d_21_00_00', 'IS_0_d_21_30_00',
        'critical_day', 'billing_day', 'Inq_1', 'Inq_2', 'Inq_3', 'Inq_4', 'Inq_5', 'Inq_6',
        'Inq_7', 'Inq_8', 'Inq_9', 'Inq_10', 'Inq_mean_7', 'Inq_std_3'
    ]]
    features_array = np.array(features).reshape(1, 1, -1)
    predicted_call = model.predict(features_array)
    predicted_call_inverse = scaler_y.inverse_transform(predicted_call.reshape(-1, 1))
    predicted_calls.append(predicted_call_inverse[0, 0])
    calc_df.loc[current_index, 'CNT_CALLS'] = predicted_call_inverse
    for i in range(1, 11):
        calc_df.loc[:, f'Inq_{i}'] = calc_df['CNT_CALLS'].shift(i)
    calc_df.loc[:, 'Inq_mean_7'] = calc_df['Inq_1'].rolling(window=7).mean()
    calc_df.loc[:, 'Inq_std_3'] = calc_df['Inq_1'].rolling(window=3).std()
    current_index += pd.Timedelta(minutes=30)

print(predicted_calls)

In [ ]:
# Filter and round predictions
calc_df.index = pd.to_datetime(calc_df.index)
filtered_df = calc_df.between_time('08:00', '21:30')
filtered_df['CNT_CALLS'] = filtered_df['CNT_CALLS'].round()
filtered_df.to_excel('calc_df LSTM октябрь.xlsx', index=True)

In [ ]:
# Plot predictions
plt.figure(figsize=(20, 6))
plt.plot(calc_df[['CNT_CALLS']].sort_index(), label=['Predictions'])
plt.legend()
plt.show()

In [ ]:
# Compare with historical data
plt.figure(figsize=(20, 6))
plt.plot(calc_df[['CNT_CALLS']].sort_index(), label='Predictions')
plt.plot(final_df.index, final_df['CNT_CALLS'], label='Historical Data')
plt.legend()
plt.show()

## Operator Forecasting Using ErlangC

Calculate required operators based on predicted call volumes.

In [ ]:
# Load predicted data
calc_df = pd.read_csv('calc_df 092024.csv', sep=';', header=0, parse_dates=True)

# Create timestamp
calc_df['MONTH'] = pd.to_datetime(calc_df['MONTH'] + '-01')
calc_df['DATESTART'] = pd.to_datetime(calc_df['DATESTART'], dayfirst=True)
calc_df['RAZREZ'] = pd.to_timedelta(calc_df['RAZREZ'] + ':00')
calc_df['TIMESTAMP'] = calc_df['DATESTART'] + calc_df['RAZREZ']
calc_df.set_index('Unnamed: 0', inplace=True)

# Prepare DataFrame for operators
for_oper = calc_df[['CNT_CALLS']].copy()
for_oper['CNT_CALLS'] = for_oper['CNT_CALLS'].str.replace(',', '.').astype(float)

In [ ]:
!pip install pyworkforce

In [ ]:
from pyworkforce.queuing import ErlangC

# Fixed parameters for ErlangC
asa = 20 / 60
aht = 3.0
interval = 30
shrinkage = 0.29
service_level = 0.8
max_occupancy = 0.75

def calculate_required_positions(transactions):
    """
    Calculate required positions using ErlangC model.
    
    Parameters:
    transactions (float): Number of transactions (calls).
    
    Returns:
    dict: Dictionary with raw_positions, positions, service_level, occupancy, waiting_probability.
    """
    erlang = ErlangC(transactions=transactions, asa=asa, aht=aht, interval=interval, shrinkage=shrinkage)
    return erlang.required_positions(service_level=service_level, max_occupancy=max_occupancy)

# Apply to DataFrame
for_oper[['raw_positions', 'positions', 'service_level', 'occupancy', 'waiting_probability']] = for_oper['CNT_CALLS'].apply(
    lambda x: pd.Series(calculate_required_positions(x))
)

for_oper.head(30)
for_oper.to_excel('for_oper октябрь.xlsx', index=True)